In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


In [97]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)
from sklearn.metrics import mean_absolute_error

In [128]:
trkm1 = pd.read_csv("train_km1.csv")
#test = pd.read_csv("tst_1dim.csv")

In [129]:
test = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/merged_test.csv")
train = pd.read_csv("/Users/seungji/Desktop/Dacon/Daesamanlap/ProcessedData/merged_train.csv")

In [130]:
tr_gr = train.groupby("단지코드").mean().reset_index()
tst_gr = test.groupby("단지코드").mean().reset_index()
# 지역 = []
# for d in train.단지코드.unique():
#     지역.append(train.loc[train.단지코드 == d,"지역"].unique()[0])
# tr_gr["지역"] = 지역
# # train["지역구분"] = [0]*len(train)
# # tr_gr.loc[tr_gr.지역.isin(['경상남도', '경기도', '전라북도', '강원도',  '충청남도','제주특별자치도',  '충청북도', '전라남도', '경상북도', '세종특별자치시']),"지역"] = "8도"
# # tr_gr.loc[tr_gr.지역.isin(['대전광역시', '광주광역시', '울산광역시', '대구광역시', '서울특별시','부산광역시']),"지역"] = "광역시"

# 자격유형 = []
# for d in train.단지코드.unique():
#     자격유형.append(train.loc[train.단지코드 == d,"자격유형"].unique()[0])
# tr_gr["자격유형"] = 자격유형

# 공급유형 = []
# for d in train.단지코드.unique():
#     공급유형.append(train.loc[train.단지코드 == d,"공급유형"].unique()[0])
# tr_gr["공급유형"] = 공급유형

# 지역 = []
# for d in test.단지코드.unique():
#     지역.append(test.loc[test.단지코드 == d,"지역"].unique()[0])
# tst_gr["지역"] = 지역
# train["지역구분"] = [0]*len(train)
# tr_gr.loc[tr_gr.지역.isin(['경상남도', '경기도', '전라북도', '강원도',  '충청남도','제주특별자치도',  '충청북도', '전라남도', '경상북도', '세종특별자치시']),"지역"] = "8도"
# tr_gr.loc[tr_gr.지역.isin(['대전광역시', '광주광역시', '울산광역시', '대구광역시', '서울특별시','부산광역시']),"지역"] = "광역시"

# 자격유형 = []
# for d in test.단지코드.unique():
#     자격유형.append(test.loc[test.단지코드 == d,"자격유형"].unique()[0])
# tst_gr["자격유형"] = 자격유형

# 공급유형 = []
# for d in test.단지코드.unique():
#     공급유형.append(test.loc[test.단지코드 == d,"공급유형"].unique()[0])
# tst_gr["공급유형"] = 공급유형

tr_gr["전용면적_std"] = train.groupby("단지코드").std().reset_index()["전용면적"]
tr_gr["전용면적별세대수_std"] = train.groupby("단지코드").std().reset_index()["전용면적별세대수"]
tr_gr["임대보증금_std"] = train.groupby("단지코드").std().reset_index()["임대보증금"]
tr_gr["임대료_std"] = train.groupby("단지코드").std().reset_index()["임대료"]
tst_gr["전용면적_std"] = test.groupby("단지코드").std().reset_index()["전용면적"]
tst_gr["전용면적별세대수_std"] = test.groupby("단지코드").std().reset_index()["전용면적별세대수"]
tst_gr["임대보증금_std"] = test.groupby("단지코드").std().reset_index()["임대보증금"]
tst_gr["임대료_std"] = test.groupby("단지코드").std().reset_index()["임대료"]
tr_re = pd.concat([train.단지코드,pd.get_dummies(train["지역"])],axis=1).groupby("단지코드").sum()
tst_re = pd.concat([test.단지코드,pd.get_dummies(test["지역"])],axis=1).groupby("단지코드").sum()
tr_re[tr_re>0] = 1
tst_re[tst_re>0] = 1
tr_gr = tr_gr.merge(tr_re, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_re, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_apart = pd.concat([train.단지코드,pd.get_dummies(train["임대건물구분"])],axis=1).groupby("단지코드").sum().reset_index()
tst_apart = pd.concat([test.단지코드,pd.get_dummies(test["임대건물구분"])],axis=1).groupby("단지코드").sum().reset_index()
tr_gr = tr_gr.merge(tr_apart, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_apart, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_gong = pd.concat([train.단지코드, pd.get_dummies(train["공급유형"])],axis =1)
tst_gong = pd.concat([test.단지코드, pd.get_dummies(test["공급유형"])],axis =1)
tr_gong["공공임대(5년,10년)"] = tr_gong["공공임대(5년)"]+tr_gong["공공임대(10년)"]
tst_gong["공공임대(5년,10년)"] = tr_gong["공공임대(10년)"]
tr_gong = tr_gong[['단지코드',   '공공임대(50년)',  '공공임대(분납)',
       '국민임대', '영구임대', '임대상가',  '행복주택', '공공임대(5년,10년)']]
tst_gong = tst_gong[['단지코드',   '공공임대(50년)',  '공공임대(분납)',
       '국민임대', '영구임대', '임대상가',  '행복주택', '공공임대(5년,10년)']]
tr_gong = tr_gong.groupby("단지코드").sum()
tst_gong = tst_gong.groupby("단지코드").sum()
tr_gong[tr_gong>0] = 1
tst_gong[tst_gong>0] = 1
tr_gong = tr_gong.reset_index()
tst_gong = tst_gong.reset_index()

tr_gr = tr_gr.merge(tr_gong, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_gong, left_on = "단지코드",right_on ="단지코드",how = "left")

tr_ja = pd.concat([train.단지코드,pd.get_dummies(train["자격유형"])],axis=1).groupby("단지코드").sum()
tst_ja = pd.concat([test.단지코드,pd.get_dummies(test["자격유형"])],axis=1).groupby("단지코드").sum()

tr_ja[tr_ja>0] = 1
tst_ja[tst_ja>0] = 1

tr_ja = tr_ja[tst_ja.columns]
tr_ja = tr_ja.reset_index()
tst_ja = tst_ja.reset_index()

tr_gr = tr_gr.merge(tr_ja, left_on = "단지코드",right_on ="단지코드",how = "left")
tst_gr =tst_gr.merge(tst_ja, left_on = "단지코드",right_on ="단지코드",how = "left")
tr_gr =tr_gr.drop("서울특별시",axis=1)

tr_gr.loc[tr_gr.전용면적_std.isna(),"전용면적_std"] = 0
tr_gr.loc[tr_gr.전용면적별세대수_std.isna(),"전용면적별세대수_std"] = 0
tr_gr.loc[tr_gr.임대보증금_std.isna(),"임대보증금_std"] = 0
tr_gr.loc[tr_gr.임대료_std.isna(),"임대료_std"] = 0

In [134]:
train = tr_gr
test = tst_gr

In [135]:
# train["등록차량수/총세대수"] = train["등록차량수"]/train["총세대수"]
train["단지내주차면수/연면적"] = train["단지내주차면수"]/train["연면적"]
train["임대보증금/임대료"] = train["임대보증금"]/train["임대료"]

# test["등록차량수/총세대수"] = test["등록차량수"]/test["총세대수"]
test["단지내주차면수/연면적"] = test["단지내주차면수"]/test["연면적"]
test["임대보증금/임대료"] = test["임대보증금"]/test["임대료"]

In [138]:
train = train[train.단지코드.isin(trkm1.단지코드)].reset_index(drop=True)

In [139]:
y = train["등록차량수"]

In [140]:
train2 = train.drop(["단지코드","위도","경도","등록차량수",'상가','C','D'],axis=1)
test2 = test.drop(["단지코드","위도","경도",'상가','C','D'],axis=1)

In [141]:
np.max(test2.loc[test2["단지내주차면수/연면적"] != np.inf,"단지내주차면수/연면적"])

119.78247051345431

In [142]:
test2.loc[test2["단지내주차면수/연면적"] == np.inf,"단지내주차면수/연면적"] = np.max(test2.loc[test2["단지내주차면수/연면적"] != np.inf,"단지내주차면수/연면적"]) 

In [143]:
train2 = train2[test2.columns]

In [144]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(train2)

X_tr = scaler.transform(train2)
X_tst = scaler.transform(test2)

In [145]:
y = train.groupby("단지코드").mean().reset_index()[["등록차량수"]]
y = np.array(y)

In [146]:
from sklearn.linear_model import LinearRegression
fit = LinearRegression().fit(X_tr,y)
tst_predict = fit.predict(X_tst)

In [147]:
tr_predict = fit.predict(X_tr)

In [148]:
mean_absolute_error(tr_predict,y)

157.52154732601642

In [149]:
pd.concat([pd.DataFrame(tr_predict),pd.DataFrame(y)],axis=1)

,0,0
0,600.4289,376.0000
1,"1,335.8659","1,665.0000"
2,840.8253,579.0000
3,"1,234.1095","1,246.0000"
4,250.8616,450.0000
5,972.2244,957.0000
6,227.7026,207.0000
7,867.9344,316.0000
8,674.3911,"1,279.0000"
9,355.1658,645.0000


In [150]:
pd.DataFrame(tst_predict)

,0
0,127.7502
1,"-1,106,196,452,445,143.6250"
2,560.7445
3,446.6733
4,-2.4223
5,253.1762
6,640.8690
7,411.8427
8,680.5964
9,441.0644


In [116]:
result = pd.concat([tst_gr.단지코드.reset_index(drop=True), pd.DataFrame(tst_predict)],axis=1)

In [117]:
result.columns = ["code","num"]

In [118]:
sample = pd.read_csv("/Users/seungji/Desktop/Dacon/parking_data/sample_submission.csv")

In [119]:
result0714 = sample.merge(result, left_on = "code", right_on = "code",how = "left")

In [120]:
result0714 = result0714[["code","num_y"]]
result0714.columns = ["code","num"]

In [122]:
result0714.loc[result0714.num<0,"num"] = np.min(result0714.loc[result0714.num>=0,"num"])

In [123]:
result0714

,code,num
0,C1072,668.1656
1,C1128,"1,168.2584"
2,C1456,481.6392
3,C1840,539.3503
4,C1332,"1,195.1990"
5,C1563,"1,655.0877"
6,C1794,913.3604
7,C1640,392.1342
8,C1377,291.3410
9,C2072,252.6911


In [124]:
result0714.to_csv("result0714.csv",index=False)